In [1]:
import pandas as pd
import numpy as np
import codecs
import re

Загружаем базу

In [2]:
scopus = pd.read_excel('Scopus_BMSTU_08042020.xlsx')

Функция, которая отделяет раздел "литература" от остальной части статьи. Просто находит заголовок и списывает все до конца.

In [3]:
#Списываем кусок от заголовка "ЛИТЕРАТУРА" до конца файла в список литературы (list)
def ref_sep (file_to_sep):
    print(file_to_sep)
    for index, line in enumerate(file_to_sep):
        if 'ЛИТЕРАТУРА' in line:
            start = index
        elif 'REFERENCE' in line.upper():
            start = index
    for i in range(start+1, len(file_to_sep)):
        references.append(file_to_sep[i])
    return references

Чтение пока работает только с текстовиками в UTF-8, пришлось для тестов пересохранять из ворда в нужной кодировке, сам перекодировать пока не умеет

In [4]:
# открываем файл, кодировка utf-8
lines = open('!eng!Серебренный_Лапин_Мокаева_ (1).txt', 'r').readlines()
references=[]

full_lines убирает из считанного файлы служебные символы

In [6]:
full_lines =[]
for i in range(len(lines)):
    lines[i] = lines[i].replace('\n', '')
    lines[i] = lines[i].replace('\t', '')
    if len(lines[i]) != 0:
        full_lines.append(lines[i])

Применяем функцию отделения списка литературы к почищенному файлу

In [7]:
ref_sep(full_lines)

['\ufeffTechnological Collaborative Robotic Systems', 'Serebrenny V.V.1, a), Lapin D.V.1, b), Mokaeva A.A.1, c), Shereuzhev M.A.1, c)', '1Bauman Moscow State Technical University, Moscow, Russia', 'c)Corresponding author: ahm.alisa@gmail.com', 'a)vsereb@bmstu.ru', 'b)lapindv@bmstu.ru', 'Annotation. The article discusses modern solutions of the assembly process automation. The Evolvable Assembly Systems (EAS) paradigm is described. EAS is aimed at creating the basis for autonomous context-sensitive and adaptable assembly systems that are able to develop with products, processes, as well as with business and social environment. Evolvability is defined as capacity of a system for adaptive evolution. In EAS, this is achieved through contextual adaptation within decentralized multi-agent control system. The features of collaborative multi-agent robotic systems paradigm are highlighted. In such systems, collaborative robots-agents (cobots) of various specialization should interact to optimiz

['1. Serebrenny, V., Shereuzhev, M., & Metasov, I. (2018). Approaches to the robotization of agricultural mobile machines. In MATEC Web of Conferences (Vol. 161, p. 03014). EDP Sciences. ',
 '2. Postanovlenie Pravitel\'stva Rossijskoj Federacii ot 18 aprelja 2016 g. N 317 «O realizacii nacional\'noj tehnologicheskoj iniciativy». [Resolution of the Government of the Russian Federation of April 18, 2016 N 317 "On the implementation of the national technology initiative"]. Retrieved from http://www.nti2035.ru/documents/Normative/ (accessed 25.12.2018). (In Russ.).',
 '3. Kagermann, H., Wahlster, W., & Helbig, J. (2013). Umsetzungsempfehlungen für das Zukunftsprojekt Industrie 4.0. Abschlussbericht des Arbeitskreises Industrie, 4(5).',
 '4. Wang, L., Törngren, M., & Onori, M. (2015). Current status and advancement of cyber-physical systems in manufacturing.\xa0Journal of Manufacturing Systems,\xa037, 517-527.',
 '5. C Chase, R. B., Aquilano, N. J., & Jacobs, F. R. (1998).\xa0Production and

Ищем в списке литературы любое упоминанение года 16-20

In [84]:
# поиск актуальных источников
fresh=[]
for line in references:
        if '2016' in line:
            fresh.append(line)
        elif '2017' in line:
            fresh.append(line)
        elif '2018' in line:
            fresh.append(line)
        elif '2019' in line:
            fresh.append(line)
        elif '2020' in line:
            fresh.append(line)
for i in fresh:
    print(i)

1. Serebrenny, V., Shereuzhev, M., & Metasov, I. (2018). Approaches to the robotization of agricultural mobile machines. In MATEC Web of Conferences (Vol. 161, p. 03014). EDP Sciences. 
2. Postanovlenie Pravitel'stva Rossijskoj Federacii ot 18 aprelja 2016 g. N 317 «O realizacii nacional'noj tehnologicheskoj iniciativy». [Resolution of the Government of the Russian Federation of April 18, 2016 N 317 "On the implementation of the national technology initiative"]. Retrieved from http://www.nti2035.ru/documents/Normative/ (accessed 25.12.2018). (In Russ.).
6. Voronov, E. M., & Serov, V. A. (2019). A Coordinated Stable-Effective Compromises Based Methodology of Design and Control in Multi-object Systems. In Smart Electromechanical Systems (pp. 147-157). Springer, Cham.
7. Glushchenko, F., & Fedotova, A. (2018, October). Developing automotive production with using product lifecycle management system. In 2018 2nd School on Dynamics of Complex Networks and their Application in Intellectual Ro

Дальше если есть DOI, то ищем в базе по DOI. Но, как показывает практика, таких статей супермало, поэтому проверку по DOI можно и не проводить.

In [8]:
#есть ли в строке источника DOI, записываем в doi_to_check что будем искать в базе
doi=[]
doi_to_check=[]
for line in references:
    doi.append(line.partition(' 10.'))
for i in doi:
    if i[2]!='':
        doi_to_check.append(i[2][:-1])# [1:-2] запись DOI без пробела спереди и точки в конце
for i in range(len(doi_to_check)):
    if doi_to_check[i][-1]=='.':
        doi_to_check[i] = doi_to_check[i][:-1]
doi_to_check

[]

In [9]:
#Находим статьи по DOI и считаем количество
count=0
for i in scopus['DOI']:
    for j in range(len(doi_to_check)):
        if doi_to_check[j] in str(i):
            count+=1
print(count)

0


Вот здесь начинается самое интересное: надо отделить в строке с источником его название от фамилий и журнала.

В этом варианте беру первые 5 слов - вылетает, их может быть меньше.

In [10]:
# регулярными выражениями находим в ссылка слова, которые входят в название, берем первые 3 позиции и склеиваем
title=[]
titles_to_check=[]
for line in fresh:
    title = re.findall(r'(\w+)\b ', line)
    words = title[0]+' '+title[1]+' '+title[2]+' '+title[3]+' '+title[4]+' '+title[5]
    titles_to_check.append(words)
for i in titles_to_check:
    print(i)

NameError: name 'fresh' is not defined

В этом варианте цепляю все слова подряд, но пока работает некорректно, цепляет журналы тоже.

In [ ]:
# регулярными выражениями находим в ссылка слова, которые входят в название, берем первые 3 позиции и склеиваем
# ! цепляет журнал и издательство
title_1=[]
titles_to_check_1=[]
words_1 = ''
for line in fresh:
    title_1 = re.findall(r'(\w+)\b ', line)
    words_1 = ' '.join(title_1)
    titles_to_check_1.append(words_1)
print(titles_to_check_1)

Это пока для проверки, что действительно эти названия вошли. Бывает, что первые 4-5 слов совпадают в нескольких статьях из базы и он один источник засчитывает за 2-3. Решится, когда разберемся с корректным отделением названий.

In [ ]:
#поиск количества вхождений названия в столбец в базе
count_tit=0
for i in scopus['Название']:
    for line in titles_to_check:
        if line in str(i):
            print(line)
            count_tit+=1
            print(count_tit,' ', i,'\n')
count_tit

Это пока не важно, штука-считалка, которая сможет потом импортировать итеративно все файлы в папке одним списком, не поштучно.

In [ ]:
import requests
gen_path='https://stepic.org/media/attachments/course67/3.6.3/'
with open('dataset_3378_3.txt') as path_in:
    url = path_in.readline().strip()
    start = gen_path + url    
    r=requests.get(start)
while 1:
        comm = r.text
        if 'We' in comm:
            print(r.text)
            break
        else:
            url_next= gen_path + comm
            print(comm)